In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
search= TavilySearchResults()

In [ ]:
search.invoke("What is the status of olympics 2024?")

### We have one tool created.
* We will create one more tool which is a retriever which retrieves data from an external source

### Retriever
* We will create retriever with our own data

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
## Load data from a website 
loader= WebBaseLoader("https://docs.smith.langchain.com/overview")
docs= loader.load()

In [ ]:
### Split the text into chunks
documents= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)

In [ ]:
### Push docs to Vector DB
vector= FAISS.from_documents(documents, OpenAIEmbeddings())
retriever= vector.as_retriever()

In [ ]:
from pprint import pprint

In [ ]:
pprint(retriever)

In [ ]:
retriever.invoke("How to upload dataset?")

#### Now this is a retriever.We will create retriever tool

In [ ]:
from langchain.tools.retriever import create_retriever_tool

In [ ]:
retriever_tool= create_retriever_tool(
    retriever,
    "langsmith_search",
    "Provides information about langsmith and its components"
)

### Now we have got Tools required. 
* Search tool
* Vector database

Now we will create agent with these two tools. But before that we will create list for tools

In [ ]:
tools= [search, retriever_tool]

### We have got the tools, now we will create agent
* To create agent we will append LLM which takes input and decides which tool is suitable to answer
Below are the requirements
* LLM
* Tools
* Prompt

In [ ]:
from langchain_openai import ChatOpenAI
llm= ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

#### Load sample prompt

In [ ]:
from langchain import hub
##Sample prompt
prompt= hub.pull("hwchase17/openai-functions-agent")
prompt.messages

### Create Agent now.

In [ ]:
from langchain.agents import create_tool_calling_agent
agent= create_tool_calling_agent(llm, tools, prompt)

### Agent Created!
### Now we have to execute the agent, for that we will call agent executer

In [ ]:
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent, tools=tools, verbose=True)

#### Now we have Agent executor. We will invoke a message and check how will it react

In [ ]:
agent_executor.invoke({"input": "Hello!"})

In [ ]:
agent_executor.invoke({"input": "How to make egg omlette?"})

In [ ]:
agent_executor.invoke({"input": "How to create langsmith dataset?"})

### Now LLM will automatically assign the tasks to respective tool.

### Lets Add memory to the agent to give better understanding of the context

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
message_history= ChatMessageHistory()

In [ ]:
agent_with_chat_history= RunnableWithMessageHistory(
    agent_executor,
    ### Session id is required for users
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
agent_with_chat_history.invoke({"input":"Hello I'm Vicky"},
                               config={"configurable": {"session_id":"Vicky"}})

In [ ]:
agent_with_chat_history.invoke({"input":"What is my name?"},
                               config={"configurable": {"session_id":"Vicky"}})

In [ ]:
agent_with_chat_history.invoke({"input":"How many medals did India won in olympics 2024?"},
                               config={"configurable": {"session_id":"Vicky"}})